## Sentiwordnet ohne Entfernen von Stopwords und Satzzeichen

In [2]:
import numpy as np # for array, linear algebra
import pandas as pd # for data processing
import matplotlib.pyplot as plt # to crreate stat and visualize data
import matplotlib as mpl  # to visualize data
import seaborn as sns # to visualize data
import matplotlib.colors as mcolors # to visualize colors
import string # collection of alphabets, words or other characters
import re # regular expression support
import nltk 
from nltk.corpus import stopwords
from string import punctuation 
# import gensim # representing documents as semantic vectors

from nltk.corpus import sentiwordnet as swn
nltk.download('sentiwordnet')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /Users/I570078/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/I570078/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/I570078/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/I570078/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

Einstellung, um kompletten Inhalt der Spalte 'Text' anzuzeigen.

In [3]:
pd.set_option('display.max_colwidth', None)

Einlesen der Daten

In [4]:
df_sentiwordnet_without_stopwords = pd.read_csv('../../Data-Preparation/traindata_with_stopwords_preprocessed.csv')

Überprüfung des Datensatzes

In [5]:
df_sentiwordnet_without_stopwords['target'].value_counts()

target
4    640000
0    640000
Name: count, dtype: int64

In [12]:
df_sentiwordnet_without_stopwords.head()

,Text,target,Ground_Truth_Label,Sentiwordnet_Classification
0,Rise and shining lol. I'm up a few mins later then planned but no rush i'll be out the door in time,4,positive,positive
1,Had a 2nd interview today. It's looking promising.,4,positive,positive
2,@EmilyAlbracht I feel your pain!,0,negative,positive
3,http://twitpic.com/6sdbj - 4yr old son and I went exploring &amp; hiking today and this is what we found Luv amphibians!,4,positive,positive
4,is lonely because all my housemates have gone to the pub,0,negative,positive


Dieser Classifier nutzt nur die Häufigste Wortbedeutung (Lemma) der jeweiligen Worte. Taucht beispielsweise das Wort "Hund" auf, wird angenommen, dass es sich um die Bedeutung des Wortes "Hund" im Sinne eines Tieres handelt und nicht etwa um eine Beleidigung. Entsprechend wird auch der Sentiment-Score bestimmt.

In [7]:
def superNaiveSentiment(review):
    reviewPolarity = 0.0
    numExceptions = 0
    words = review.lower().split()
    
    for word in words:
        weight = 0.0
        synsets = list(swn.senti_synsets(word))
        if synsets:
            common_meaning = synsets[0]
            pos_score = common_meaning.pos_score()
            neg_score = common_meaning.neg_score()
            if pos_score > neg_score:
                weight += pos_score
            elif pos_score < neg_score:
                weight -= neg_score
        else:
            numExceptions += 1
        reviewPolarity += weight
    return reviewPolarity


In [9]:
df_sentiwordnet_without_stopwords['Sentiwordnet_Classification'] = df_sentiwordnet_without_stopwords['Text'].apply(lambda x: 'positive' if superNaiveSentiment(x) >= 0 else 'negative')

In [13]:
df_sentiwordnet_without_stopwords.head()

,Text,target,Ground_Truth_Label,Sentiwordnet_Classification
0,Rise and shining lol. I'm up a few mins later then planned but no rush i'll be out the door in time,4,positive,positive
1,Had a 2nd interview today. It's looking promising.,4,positive,positive
2,@EmilyAlbracht I feel your pain!,0,negative,positive
3,http://twitpic.com/6sdbj - 4yr old son and I went exploring &amp; hiking today and this is what we found Luv amphibians!,4,positive,positive
4,is lonely because all my housemates have gone to the pub,0,negative,positive


Evaluation

In [11]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(df_sentiwordnet_without_stopwords['Ground_Truth_Label'], df_sentiwordnet_without_stopwords['Sentiwordnet_Classification'])
print(f'Accuracy: {accuracy}')

Accuracy: 0.5829796875
